In [6]:
!pip install transformers
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e41f060c549b00c4e1fe9f22026461acb79da09fe2f4b477bfec035b0de671b6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [14]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
import torch, json
import torch.nn.functional as F

with open("data_v3.json", "r") as f:
    data = json.load(f)

def str_to_bool(s):
    if s.lower() == 'true':
        return True
    elif s.lower() == 'false':
        return False
    else:
        raise ValueError("Cannot convert {} to a bool".format(s))

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def split_into_strings(text):
    sentences = text.split('.')
    sentences = [string for string in sentences if string]
    strings = [s.split(',') for s in sentences]
    filtered_list = [e for e in strings if len(e) != 1 and e[0] != '']

    list = []
    for e in strings:
      for i in e:
        list.append(i)

    return list

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [21]:
def similarity_check(criteria, answer):
  #Sentences we want sentence embeddings for
  #sentences = ['Memory allocated with malloc() must be explicitly deallocated with free().', 'memory allocated with malloc should be deallocated with free.']

  # Tokenize sentences
  encoded_input_c = tokenizer(criteria, padding=True, truncation=True, return_tensors='pt')
  encoded_input_a = tokenizer(answer, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output_c = model(**encoded_input_c)
      model_output_a = model(**encoded_input_a)

  # Perform pooling
  sentence_embeddings_c = mean_pooling(model_output_c, encoded_input_c['attention_mask'])
  sentence_embeddings_a = mean_pooling(model_output_a, encoded_input_a['attention_mask'])

  # Normalize embeddings
  sentence_embeddings_c = F.normalize(sentence_embeddings_c, p=2, dim=1)
  sentence_embeddings_a = F.normalize(sentence_embeddings_a, p=2, dim=1)

  #print("Sentence embeddings:")
  #print(sentence_embeddings)

  sim = util.pytorch_cos_sim(sentence_embeddings_c, sentence_embeddings_a)

  return sim

In [26]:
results = []
labels = []
for e in data:
  print("==========================================================================")
  evaluated = False
  res  = False
  label = str_to_bool(e['score'])
  labels.append(label)

  #criteria = split_into_strings(e['criteria'])
  criteria = e['criteria']
  answer = split_into_strings(e['answer'])

  cosine_scores = similarity_check(criteria, answer)

  print(criteria)
  print(answer)
  print("cos score:")
  print(cosine_scores)
  #print("eucl score:")
  #print(euclidean_dist_easy)
  #print(type(cosine_scores))

  for row in cosine_scores:
    for val in row:

      print(val.item())
      if(val.item() > 0.825 ):
        res = True
        evaluated = True
        break

    if(evaluated == True):
      break

  print("label: " + str(label))
  print("outcome: " + str(res))
  if (res == label):
    results.append(True)
  else:
    results.append(False)



  print("==========================================================================")


print("results len:")
print(len(results))
print(results)

print("==========================================================================")
print("===============================evaluation=================================")
print("==========================================================================")

correct_count = 0
for e in results:
  if (e == True):
    correct_count = correct_count + 1

print('\n')
print('\n')

score = correct_count / len(results)

print(score)



Memory allocated with malloc() must be explicitly deallocated with free().
['Memory allocated with new must be explicitly deallocated with free()']
cos score:
tensor([[0.7885]])
0.7884525060653687
label: False
outcome: False
Memory allocated with malloc() must be explicitly deallocated with free().
['whereas memory allocated with new should be deallocated with delete']
cos score:
tensor([[0.5606]])
0.5605571866035461
label: False
outcome: False
Memory allocated with malloc() must be explicitly deallocated with free().
['whereas memory allocated with free() should be deallocated with malloc']
cos score:
tensor([[0.8517]])
0.8516903519630432
label: False
outcome: True
Memory allocated with malloc() must be explicitly deallocated with free().
['Memory allocated with malloc() must be explicitly deallocated with delete']
cos score:
tensor([[0.8205]])
0.8205074071884155
label: False
outcome: False
Memory allocated with new must be explicitly deallocated with delete.
['whereas memory allocate